In [ ]:
!python -V

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error
import pickle
import mlflow

In [ ]:
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

In [ ]:
mlflow.set_experiment("nyc-taxi-exp")

In [ ]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')

In [ ]:
df.head()

In [ ]:
df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime

In [ ]:
df

In [ ]:
td = df.duration.iloc[0]

In [ ]:
td

In [ ]:
td.total_seconds()

In [ ]:
df.duration = df.duration.apply(lambda td: td.total_seconds()/60)

In [ ]:
sns.distplot(df.duration)

In [ ]:
df.duration.describe(percentiles=[0.9,0.95,0.96,0.97,0.98,0.99,1.0])

In [ ]:
((df.duration >=1) & (df.duration <=60)).mean()

In [ ]:
df_avgtrips = df[(df.duration >=1) & (df.duration <=60)]

In [ ]:
df_avgtrips.dtypes

In [ ]:
categorical = ['PULocationID','DOLocationID']
numerical = ['trip_distance']

In [ ]:
df_avgtrips[categorical] = df_avgtrips[categorical].astype(str)

In [ ]:
df_avgtrips.dtypes

In [ ]:
df_avgtrips[categorical + numerical].iloc[:10].to_dict(orient='records')

In [ ]:
train_dicts = df_avgtrips[categorical + numerical].to_dict(orient='records')

In [ ]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [ ]:
X_train

In [ ]:
dv.feature_names_

In [ ]:
target = 'duration'
y_train = df_avgtrips[target].values

In [ ]:
y_train

In [ ]:
lr = LinearRegression()
lr.fit(X_train,y_train)

In [ ]:
y_pred = lr.predict(X_train)

In [ ]:
sns.distplot(y_pred, label='prediction')

In [ ]:
sns.distplot(y_train, label='actual')

In [ ]:
sns.distplot(y_pred, label='prediction')
sns.distplot(y_train, label='actual')
plt.legend()

In [ ]:
mean_squared_error(y_train, y_pred, squared = False)
#squared = False to get root mean squared error value

In [ ]:
def read_dataFrame(filename):
    df = pd.read_parquet(filename)
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds()/60)
    df = df[(df.duration >=1) & (df.duration <=60)]
    categorical = ['PULocationID','DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [ ]:
df_train = read_dataFrame('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val = read_dataFrame('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

In [ ]:

df_val.dtypes

In [ ]:
categorical = ['PULocationID','DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

In [ ]:
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)


In [ ]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [ ]:
lr1 = Ridge(alpha=10)
lr1.fit(X_train,y_train)

y_pred = lr1.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)


In [ ]:
with open('./models/lin_reg.bin','wb') as f:
    pickle.dump((dv,lr1), f)

In [ ]:
with mlflow.start_run():
    mlflow.set_tag('developer', 'arungansi')
    
    mlflow.log_param('train_data_path','green_tripdata_2021-01.parquet')
    mlflow.log_param('val_data_path','green_tripdata_2021-02.parquet')
    # mlflow.log_input('train_data_path','green_tripdata_2021-01.parquet')
    # mlflow.log_input('train_data_path','green_tripdata_2021-01.parquet')
    
    alpha = 10
    mlflow.log_param('alpha',alpha)
    lr1 = Ridge(alpha)
    lr1.fit(X_train,y_train)

    y_pred = lr1.predict(X_val)

    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric('rmse',rmse)
    
    mlflow.log_artifact(local_path='models/lin_reg.bin', artifact_path='models_pickle')

In [ ]:
import xgboost as xgb


In [ ]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [ ]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [ ]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

Best Parameters after 1000 runs is the one with the lowest rmse
Choose the hyper paramters for that run and train the model again at the end of the hypertuning run above

In [ ]:
params = {
    'learning_rate':0.07755880320609634,
    'max_depth':10,
    'min_child_weight':3.7126270987128547,
    'objective':'reg:linear',
    'reg_alpha':0.028088122108329037,
    'reg_lambda':0.013125534079775656,
    'seed':42
}

In [ ]:
mlflow.xgboost.autolog()

In [ ]:
booster = xgb.train(
    params=params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, 'validation')],
    early_stopping_rounds=50
)

In [ ]:
mlflow.xgboost.autolog(disable=True)

In [ ]:
with mlflow.start_run():
    params = {
    'learning_rate':0.07755880320609634,
    'max_depth':10,
    'min_child_weight':3.7126270987128547,
    'objective':'reg:linear',
    'reg_alpha':0.028088122108329037,
    'reg_lambda':0.013125534079775656,
    'seed':42
    }
    
    mlflow.log_params(params)
    
    booster = xgb.train(
    params=params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, 'validation')],
    early_stopping_rounds=50
    )
    
    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val,y_pred,squared=False)
    mlflow.log_metric('rmse',rmse)
    
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv,f_out)
        
    mlflow.log_artifact("models/preprocessor.b",artifact_path='preprocessor')
    
    mlflow.xgboost.log_model(booster,artifact_path="models_mlflow")
    

In [ ]:
logged_model = 'runs:/d031ab694f7f4a76988537cd5f7594ce/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)



In [ ]:
loaded_model

In [ ]:
X_val

In [ ]:
# Predict on a Pandas DataFrame.
#loaded_model.predict(pd.DataFrame(<data>))

In [45]:
#2nd method

xgboost_loaded_model = mlflow.xgboost.load_model(logged_model)

/home/codespace/anaconda3/envs/exp-tracking/lib/python3.12/site-packages/xgboost/core.py:160: UserWarning: [01:37:52] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [46]:
xgboost_loaded_model

In [47]:
y_pred = xgboost_loaded_model.predict(valid)

In [48]:
y_pred

array([15.088369 ,  6.6498303, 17.523094 , ..., 12.988363 ,  6.5416555,
        8.390461 ], dtype=float32)